![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Limpieza de datos para crear gráficos descriptivos y modelo de ML

Nos dan un dataset con información relativa da distintas películas y el número de likes que han recibido en Facebook y Twitter. Nos piden:

1. Preparar el dataset para crear un modelo de ML que nos permita predecir el número de likes a partir de las características de las películas.
2. Además del modelo, necesitaremos presentar algunos gráficos a nuestros stakeholders. Es necesario que la data este preparada para ello.
3. Identificar problemas graves en la data y comunicarlos lo antes posible.
4. Adicionalmente, nuestro stakeholder tiene "curiosidad" por algunas cifras:
    - Actores más repetidos en el dataset
    - Géneros de películas más comunes
    - Relación entre presupuesto y recaudación


In [72]:
import pandas as pd
import numpy as np


In [73]:
#??pd.read_csv

In [74]:
df = pd.read_csv("data/imdb_clase.csv", sep=';', index_col= 0).reset_index(drop = True)#index 0 porque la columna 0 es el indice ,si tenemos una columna unnamed, y no la queremos porque el programa la refleja como una cosa rara

## Leer datos

## Limpieza de datos

### Revisiones generales: `.columns`, `.index`, `.shape`

In [75]:
df.columns

Index(['color', 'director_name', 'duration', 'gross', 'genres', 'movie_title',
       'title_year', 'language', 'country', 'budget', 'imdb_score', 'actors',
       'movie_facebook_likes', 'movie_twitter_likes'],
      dtype='object')

In [76]:
df.describe(include='all').T#nos fimjamos en los negativos del min, el include incluye no solo las numericas, sino tambien las categoricas. Nan es valor vacio

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
color,88,3,Color,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
director_name,88,63,Ridley Scott,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,99.0,NaN,NaN,NaN,155.494949,72.797927,-50.0,138.5,143.0,155.0,650.0
gross,99,87,$nan,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genres,98,56,Action|Adventure|Sci-Fi,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
movie_title,99,91,ben-hur,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title_year,99.0,NaN,NaN,NaN,1944.010101,393.848787,-1000.0,2012.0,2013.0,2014.0,2030.0
language,99,1,English,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country,99,12,USA,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
budget,99,57,"USD 100,000,000.00",6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df.index

RangeIndex(start=0, stop=99, step=1)

In [78]:
df.shape

(99, 14)

### Ver algunas observaciones: `.head()`, `.tail()`, `.sample()`

In [79]:
#df.head().T Si ponemos .T lo trasponemos
df.head()

,color,director_name,duration,gross,genres,movie_title,title_year,language,country,budget,imdb_score,actors,movie_facebook_likes,movie_twitter_likes
0,Color,Martin Scorsese,240,"$116,866,727.00",Biography|Comedy|Crime|Drama,the wolf of wall street,2013,English,USA,"USD 100,000,000.00",8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000,1000
1,Color,Shane Black,195,"$408,992,272.00",Action|Adventure|Sci-Fi,iron man 3,2013,English,USA,"USD 200,000,000.00",7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000,1000
2,color,Quentin Tarantino,187,"$54,116,191.00",Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,English,USA,"USD 44,000,000.00",7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000,1000
3,Color,Kenneth Lonergan,186,"$46,495.00",Drama,margaret,2011,English,usa,"USD 14,000,000.00",6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0,1000
4,Color,Peter Jackson,186,"$258,355,354.00",Adventure|Fantasy,the hobbit: the desolation of smaug,2013,English,USA,"USD 225,000,000.00",7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000,1000


In [80]:
df.sample(5)

,color,director_name,duration,gross,genres,movie_title,title_year,language,country,budget,imdb_score,actors,movie_facebook_likes,movie_twitter_likes
43,Color,Tate Taylor,146,"$169,705,587.00",Drama,the help,2011,English,USA,"USD 25,000,000.00",8.1,"Emma Stone,Bryce Dallas Howard,Mike Vogel",75000,1000
3,Color,Kenneth Lonergan,186,"$46,495.00",Drama,margaret,2011,English,usa,"USD 14,000,000.00",6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0,1000
79,Color,Ridley Scott,138,"$16,969,390.00",Crime|Drama|Thriller,the counselor,2013,English,USA,"USD 25,000,000.00",5.3,"Michael Fassbender,Brad Pitt,Goran Visnjic",24000,1000
65,NaN,Oliver Stone,141,"$47,307,550.00",Crime|Drama|Thriller,savages,2012,English,USA,"USD 45,000,000.00",6.5,"Demián Bichir,Shea Whigham,Gary Stretch",28000,1000
4,Color,Peter Jackson,186,"$258,355,354.00",Adventure|Fantasy,the hobbit: the desolation of smaug,2013,English,USA,"USD 225,000,000.00",7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000,1000


### Describir dataset: `info()`, `describe()`

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   color                 88 non-null     object 
 1   director_name         88 non-null     object 
 2   duration              99 non-null     int64  
 3   gross                 99 non-null     object 
 4   genres                98 non-null     object 
 5   movie_title           99 non-null     object 
 6   title_year            99 non-null     int64  
 7   language              99 non-null     object 
 8   country               99 non-null     object 
 9   budget                99 non-null     object 
 10  imdb_score            99 non-null     float64
 11  actors                99 non-null     object 
 12  movie_facebook_likes  99 non-null     int64  
 13  movie_twitter_likes   99 non-null     int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 11.0+ KB


### Valores faltantes: `.isnull()`, `.isna()`

In [82]:
df.isnull().any()#con any va columna a columna diciendo si hay algun null, y si es asi te pone true

color                    True
director_name            True
duration                False
gross                   False
genres                   True
movie_title             False
title_year              False
language                False
country                 False
budget                  False
imdb_score              False
actors                  False
movie_facebook_likes    False
movie_twitter_likes     False
dtype: bool

In [83]:
df.isna().sum()#los na en cada columna

color                   11
director_name           11
duration                 0
gross                    0
genres                   1
movie_title              0
title_year               0
language                 0
country                  0
budget                   0
imdb_score               0
actors                   0
movie_facebook_likes     0
movie_twitter_likes      0
dtype: int64

In [84]:
df.isna().mean()*100#en porcentaje

color                   11.111111
director_name           11.111111
duration                 0.000000
gross                    0.000000
genres                   1.010101
movie_title              0.000000
title_year               0.000000
language                 0.000000
country                  0.000000
budget                   0.000000
imdb_score               0.000000
actors                   0.000000
movie_facebook_likes     0.000000
movie_twitter_likes      0.000000
dtype: float64

In [85]:
df[df.duplicated(keep = False)]#filas duplicadas, y el keep false, lo que hace es te saca las filas duplicadas de arriba abajo

,color,director_name,duration,gross,genres,movie_title,title_year,language,country,budget,imdb_score,actors,movie_facebook_likes,movie_twitter_likes
8,Color,Joss Whedon,173,"$623,279,547.00",Action|Adventure|Sci-Fi,the avengers,2012,English,USA,"USD 220,000,000.00",8.1,"Chris Hemsworth,Robert Downey Jr.,Scarlett Joh...",123000,1000
9,Color,Joss Whedon,173,"$623,279,547.00",Action|Adventure|Sci-Fi,the avengers,2012,English,USA,"USD 220,000,000.00",8.1,"Chris Hemsworth,Robert Downey Jr.,Scarlett Joh...",123000,1000
30,Color,Ridley Scott,150,"$65,007,045.00",Action|Adventure|Drama,exodus: gods and kings,2014,English,UK,"USD 140,000,000.00",6.1,"Christian Bale,María Valverde,Ben Mendelsohn",51000,1000
31,Color,Ridley Scott,150,"$65,007,045.00",Action|Adventure|Drama,exodus: gods and kings,2014,English,UK,"USD 140,000,000.00",6.1,"Christian Bale,María Valverde,Ben Mendelsohn",51000,1000
52,Color,Sam Mendes,143,"$304,360,277.00",Action|Adventure|Thriller,skyfall,2012,English,UK,"USD 200,000,000.00",7.8,"Albert Finney,Helen McCrory,Rory Kinnear",80000,1000
53,Color,Sam Mendes,143,"$304,360,277.00",Action|Adventure|Thriller,skyfall,2012,English,UK,"USD 200,000,000.00",7.8,"Albert Finney,Helen McCrory,Rory Kinnear",80000,1000
54,Color,Baz Luhrmann,143,"$144,812,796.00",Drama|Romance,the great gatsby,2013,English,Australia,"USD 105,000,000.00",7.3,"Leonardo DiCaprio,Elizabeth Debicki,Steve Bisley",115000,1000
55,Color,Baz Luhrmann,143,"$144,812,796.00",Drama|Romance,the great gatsby,2013,English,Australia,"USD 105,000,000.00",7.3,"Leonardo DiCaprio,Elizabeth Debicki,Steve Bisley",115000,1000
61,Color,Timur Bekmambetov,141,$nan,Adventure|Drama|History,ben-hur,2016,English,USA,"USD 100,000,000.00",6.1,"Morgan Freeman,Ayelet Zurer,Moises Arias",0,1000
63,Color,Timur Bekmambetov,141,$nan,Adventure|Drama|History,ben-hur,2016,English,USA,"USD 100,000,000.00",6.1,"Morgan Freeman,Ayelet Zurer,Moises Arias",0,1000


In [86]:
df.duplicated().value_counts()

False    94
True      5
dtype: int64

In [87]:
df.shape, df.drop_duplicates().shape

((99, 14), (94, 14))

In [88]:
df = df.drop_duplicates()

### Quitar filas duplicadas: `.duplicated()`. `.drop_duplicates()`

## Tratamiento de columnas

Dylan:

- `color`
- `director_name`
- `gross`
- `duration`

Todos:
- `movie_title`
- `title_year`
- `movie_twitter_likes`
- `imdb_score`
- `country`
- `language`
- `actors`
- `movie_facebook_likes`


### Columna `color`

In [89]:
df['color'].value_counts()#si volvemos al df.shape vemos que hay 94 filas, para que nos enseñe los nulos, celda de abajo

Color               81
color                1
 Black and White     1
Name: color, dtype: int64

In [90]:
df['color'].value_counts(dropna=False)

Color               81
NaN                 11
color                1
 Black and White     1
Name: color, dtype: int64

In [91]:
df[df['color'].isna()].T

,5,10,15,18,56,65,74,76,80,83,87
color,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
director_name,NaN,Tom Tykwer,Richard Linklater,Christopher Nolan,NaN,Oliver Stone,Terrence Malick,Robert Zemeckis,James Mangold,Walter Salles,Seth MacFarlane
duration,183,172,165,164,143,141,139,138,138,137,136
gross,"$330,249,062.00","$27,098,580.00","$25,359,200.00","$448,130,642.00",$nan,"$47,307,550.00","$13,303,319.00","$93,749,203.00","$132,550,960.00","$717,753.00","$42,615,685.00"
genres,Action|Adventure|Sci-Fi,Drama|Sci-Fi,Drama,Action|Thriller,Drama|Horror|Thriller,Crime|Drama|Thriller,Drama|Fantasy,Drama|Thriller,Action|Adventure|Sci-Fi|Thriller,Adventure|Drama,Comedy|Western
movie_title,batman v superman: dawn of justice,cloud atlas,boyhood,the dark knight rises,the ridges,savages,the tree of life,flight,the wolverine,on the road,a million ways to die in the west
title_year,202,2012,2014,2012,2011,2012,2011,2012,2013,2012,2014
language,English,English,English,English,English,English,English,English,English,English,English
country,USA,Germany,USA,USA,USA,USA,USA,USA,USA,France,USA
budget,"USD 250,000,000.00","USD 102,000,000.00","USD 4,000,000.00","USD 250,000,000.00","USD 17,350.00","USD 45,000,000.00","USD 32,000,000.00","USD 31,000,000.00","USD 120,000,000.00","USD 25,000,000.00","USD 40,000,000.00"


In [92]:
df.loc[df['color'].isna(),'color'] = 'Color'
df

,color,director_name,duration,gross,genres,movie_title,title_year,language,country,budget,imdb_score,actors,movie_facebook_likes,movie_twitter_likes
0,Color,Martin Scorsese,240,"$116,866,727.00",Biography|Comedy|Crime|Drama,the wolf of wall street,2013,English,USA,"USD 100,000,000.00",8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000,1000
1,Color,Shane Black,195,"$408,992,272.00",Action|Adventure|Sci-Fi,iron man 3,2013,English,USA,"USD 200,000,000.00",7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000,1000
2,color,Quentin Tarantino,187,"$54,116,191.00",Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,English,USA,"USD 44,000,000.00",7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000,1000
3,Color,Kenneth Lonergan,186,"$46,495.00",Drama,margaret,2011,English,usa,"USD 14,000,000.00",6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0,1000
4,Color,Peter Jackson,186,"$258,355,354.00",Adventure|Fantasy,the hobbit: the desolation of smaug,2013,English,USA,"USD 225,000,000.00",7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Color,Steve McQueen,134,"$56,667,870.00",Biography|Drama|History,12 years a slave,2013,English,USA,"USD 20,000,000.00",8.1,"Quvenzhané Wallis,Scoot McNairy,Taran Killam",83000,1000
95,Color,Richard J. Lewis,134,"$7,501,404.00",Comedy|Drama,barney's version,2010,English,Canada,USD nan,7.3,"Mark Addy,Atom Egoyan,Paul Gross",0,1000
96,Color,Paul Greengrass,134,"$107,100,855.00",Biography|Drama|Thriller,captain phillips,2013,English,USA,"USD 55,000,000.00",7.9,"Tom Hanks,Chris Mulkey,Michael Chernus",65000,1000
97,Color,David Ayer,134,"$85,707,116.00",Action|Drama|War,fury,2014,English,USA,"USD 68,000,000.00",7.6,"Brad Pitt,Logan Lerman,Jim Parrack",82000,1000


In [93]:
df['color'].value_counts()

Color               92
color                1
 Black and White     1
Name: color, dtype: int64

In [94]:
df['color'].str.capitalize().value_counts()#el black and white, python ha puesto el tabulado en mayuscula

Color               92
Color                1
 black and white     1
Name: color, dtype: int64

In [95]:
df['color'] = df['color'].str.strip().str.capitalize()#el strip limpia los espacios de delante y de despues y luego pone la primera letra en mayuscula

In [97]:
#assert si lo que viene despues de assert es True no pasa nada pero si es False, da un error.
assert df['color'].isna().sum() == 0 #aqui le decimos una condicion, si la suma de nan es = 0, pues entonces corre bien, si hubiera algun nulo nos saltaba un error
assert df['color'].unique().tolist() == ['Color','Black and white']

### Columna `director_name`

In [100]:
df['director_name'].isna().mean()

Michael Bay          3
Peter Jackson        3
Christopher Nolan    3
Ridley Scott         3
Martin Scorsese      2
                    ..
Adam McKay           1
Zack Snyder          1
Baz Luhrmann         1
Kenneth Lonergan     1
Clint Eastwood       1
Name: director_name, Length: 63, dtype: int64

In [106]:
pd.set_option('display.max_rows', None)#none es lo que hace que nos muestren todas las columnas

In [104]:
df['director_name'].value_counts(dropna=False)

NaN                  11
Michael Bay           3
Peter Jackson         3
Christopher Nolan     3
Ridley Scott          3
                     ..
Adam McKay            1
Zack Snyder           1
Baz Luhrmann          1
Kenneth Lonergan      1
Clint Eastwood        1
Name: director_name, Length: 64, dtype: int64

In [110]:
df.loc[df['director_name'].isna(),'director_name'] = 'No informacion'


In [113]:
assert df['director_name'].isna().sum()==0

### Columna `gross`

### Columna `genres`


### Columna `budget`

### Columna `duration`

## Guardar dataset

### Columna `movie_title`

### Columna `title_year`

### Columna `movie_twitter_likes`

### Columna `imdb_score`

### Columna `country`

### Columna `language`

### Columna `movie_facebook_likes`

### Columna `actors`